# Mediator

## O que é?

O _Mediator_ prevê a ciração de um objecto central que serve para gerenciar a interação de multiplos objetos.

## Por quê?

Muitas vezes, objetos precisam ter conhecimento e interagir uns cons os outros, issos os torna difíceis de reutilisar e aumenta complexidade do grupo inteiro cada vez que um novo elemento é adicionado. Utilizando o _Mediator_ os objectos tem apenas que se comunicar com um único objeto (o _mediator_ ), tornando-os desacoplados e limitando a complexidade ao _mediator_ .

## Estrutura

![mediator](assets/mediator.png)

## Exemplo

Nossa empresa tem um serviço de ingestão de dados através de uma API _restful_, para eventos online, e um sistema de ingestão em massa. Ambos devem processar os dados e armazená-los em uma database transacional, enquanto um serviço de _logging_ audita as transações.

Vejamos uma possível implementação.

In [6]:
import datetime as dt

class OnlineAPI:
    """
    Online events handler.
    """
    
    def __init__(self, db, logger):
        self._db = db
        self._logger = logger
        
    def put(self, msg):
        """
        Forwards the message to storage.
        """
        self._logger.info(self)
        self._db.store(msg)
     
    
class OfflineIngester:
    """
    Batch events handler.
    """
    def __init__(self, db, logger):
        self._db = db
        self._logger = logger
        
    def ingest(self, *records):
        """
        Reads multiple records and store them.
        """
        for record in records:
            self._logger.info(self)
            self._db.store(record)
    
    
class DB:
    """
    Simple database;
    """
    def __init__(self, logger):
        self._logger = logger
        self.storage = []
        
    def store(self, struct):
        """
        Persists the structure (not really).
        """
        self._logger.info(self)
        self.storage.append(struct)
        

class Logger:
    """
    For all your logging needs.
    """
    def __init__(self):
        self.logs = []
        
    def info(self, origin):
        """
        Create an INFO level record with timestamps.
        """
        time = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        self.logs.append(f"INFO: {origin.__class__.__name__} - {time}")

In [7]:
logger = Logger()
db = DB(logger)
offline = OfflineIngester(db, logger)
online = OnlineAPI(db, logger)

In [8]:
offline.ingest({"id": 1, "name": "Pedro"}, {"id": 2, "name": "Paulo"})
online.put({"id": 3, "name": "Judas"})

In [9]:
db.storage

[{'id': 1, 'name': 'Pedro'},
 {'id': 2, 'name': 'Paulo'},
 {'id': 3, 'name': 'Judas'}]

In [10]:
logger.logs

['INFO: OfflineIngester - 2020-07-05T17:31:08',
 'INFO: DB - 2020-07-05T17:31:08',
 'INFO: OfflineIngester - 2020-07-05T17:31:08',
 'INFO: DB - 2020-07-05T17:31:08',
 'INFO: OnlineAPI - 2020-07-05T17:31:08',
 'INFO: DB - 2020-07-05T17:31:08']

Podemos ver que todas classes têm conhecimento umas da outras, assim com têm referências a instâncias e utilizam um interface bem específica.

Nesse caso, um _mediator_ pode ser utilizado para promover o desacoplamento dessas classes.

In [21]:
import abc


class Mediator:
    """
    Manages the interaction between the different objects that compose the system.
    """
    def create_colleagues(self):
        self._online = OnlineAPI(self)
        self._offline = OfflineIngester(self)
        self._logger = Logger(self)
        self._db = DB(self)
        
        return self._online, self._offline, self._logger, self._db
    
    def notify(self, origin, msg):
        """
        Handles events generated by one of the agents in this system.
        """
        if origin != self._logger:
            self._logger.info(origin)
            
        if origin in (self._online, self._offline):
            self._db.store(msg)

            
class Colleague(metaclass=abc.ABCMeta):
    """
    Simple agent of the system.
    """
        
    def __init__(self, mediator):
        self._mediator = mediator
        
    def notify(self, msg):
        """
        Notify mediator of an event.
        """
        self._mediator.notify(self, msg)
            

class OnlineAPI(Colleague):
    """
    Online events handler.
    """
    def put(self, msg):
        """
        Forwards the message to storage.
        """
        self.notify(msg)
     
    
class OfflineIngester(Colleague):
    """
    Batch events handler.
    """
    def ingest(self, *records):
        """
        Reads multiple records and store them.
        """
        for record in records:
            self.notify(record)
    
    
class DB(Colleague):
    """
    Simple database;
    """
    
    def __init__(self, mediator):
        super().__init__(mediator)
        self.storage = []
        
    def store(self, struct):
        """
        Persists the structure (not really).
        """
        self.storage.append(struct)
        self.notify(struct)
        
        
class Logger(Colleague):
    """
    For all your logging needs.
    """
    def __init__(self, mediator):
        super().__init__(mediator)
        self.logs = []
        
    def info(self, origin):
        """
        Create an INFO level record with timestamps.
        """
        time = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        self.logs.append(f"INFO: {origin.__class__.__name__} - {time}")

Repetindo o procedimento anterior.

In [22]:
mediator = Mediator()
online, offline, logger, db = mediator.create_colleagues()

In [23]:
offline.ingest({"id": 1, "name": "Pedro"}, {"id": 2, "name": "Paulo"})
online.put({"id": 3, "name": "Judas"})

In [24]:
db.storage

[{'id': 1, 'name': 'Pedro'},
 {'id': 2, 'name': 'Paulo'},
 {'id': 3, 'name': 'Judas'}]

In [25]:
logger.logs

['INFO: OfflineIngester - 2020-07-05T17:32:39',
 'INFO: DB - 2020-07-05T17:32:39',
 'INFO: OfflineIngester - 2020-07-05T17:32:39',
 'INFO: DB - 2020-07-05T17:32:39',
 'INFO: OnlineAPI - 2020-07-05T17:32:39',
 'INFO: DB - 2020-07-05T17:32:39']

Como podemos ver as classes não mais têm conhecimento umas das outras, apenas do mediador. Além disso, a função de _logging_ veio quase de graça.

Por outro lado, podemos ver que o method `notify` do _mediator_ pode crescer rapidamente e tornar-se bastante complexo. Talvez um bom meio termo seja a utilização de sistema de _pub-sub_ .

In [52]:
from collections import defaultdict

class PubSubMediator:
    """
    Mediator that notifies events to all colleagues subscribed to a topic.
    """
    def __init__(self):
        self.topics = defaultdict(list)
    
    def subscribe(self, colleague, topic):
        """
        Subscribe colleague to given topic.
        """
        self.topics[topic].append(colleague)
    
    def notify(self, colleague, msg, *topics):
        """
        Notify all colleagues subscribed to `topics`.
        """
        for topic in topics:
            for subscriber in self.topics[topic]:
                if subscriber == colleague:
                    continue
                subscriber.notified(colleague, msg)

            
class Colleague(metaclass=abc.ABCMeta):
        
    def __init__(self, mediator):
        self._mediator = mediator
        
    def notify(self, msg, *topics):
        """
        Notify the mediator.
        """
        self._mediator.notify(self, msg, *topics)
        
    def subscribe(self, topic):
        """
        Subscribe to a topic.
        """
        self._mediator.subscribe(self, topic)
        
    def notified(self, origin, msg):
        """
        Get a notification that an event has happened.
        """
        pass
            

class OnlineAPI(Colleague):
    def put(self, msg):
        self.notify(msg, "ingest", "log")
     
    
class OfflineIngester(Colleague):
    def ingest(self, *records):
        for record in records:
            self.notify(record, "ingest", "log")
    
    
class DB(Colleague):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.storage = []
        
    def store(self, struct):
        self.storage.append(struct)
        self.notify({}, "log")
        
    def notified(self, origin, msg):
        self.store(msg)
        
        
class Logger(Colleague):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.logs = []
        
    def info(self, origin):
        time = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        self.logs.append(f"INFO: {origin.__class__.__name__} - {time}")
        
    def notified(self, origin, msg):
        self.info(origin)

In [53]:
mediator = PubSubMediator()

logger = Logger(mediator)
db = DB(mediator)
offline = OfflineIngester(mediator)
online = OnlineAPI(mediator)

In [54]:
# subscribing to topics

logger.subscribe("log")
db.subscribe("ingest")

In [55]:
offline.ingest({"id": 1, "name": "Pedro"}, {"id": 2, "name": "Paulo"})
online.put({"id": 3, "name": "Judas"})

In [56]:
db.storage

[{'id': 1, 'name': 'Pedro'},
 {'id': 2, 'name': 'Paulo'},
 {'id': 3, 'name': 'Judas'}]

In [57]:
logger.logs

['INFO: DB - 2020-07-05T18:47:33',
 'INFO: OfflineIngester - 2020-07-05T18:47:33',
 'INFO: DB - 2020-07-05T18:47:33',
 'INFO: OfflineIngester - 2020-07-05T18:47:33',
 'INFO: DB - 2020-07-05T18:47:33',
 'INFO: OnlineAPI - 2020-07-05T18:47:33']

Podemos ver que nesse caso a responsabilidade de gerir a interação entre os elementos é dividida entre o _mediator_ e os _colleagues_ . O mediator é reponsável por gerir os tópicos, enquanto os _colleagues_ são repossáveis por saber quais tópicos são relevantes para eles. Dessa forma, os _colleagues_ não precisam conhecer a interface de outras classes ou com quais outros objetos eles estão interagindo.

## Prós e contras:

### Pros
- Permitem que as classes tornem-se desacopladas umas das outras, resultando maior reutilização de código e classes mais simples
- Instanciar classes torna-se mais fácil
- Pelo fato do mediador ser o único ponto de troca de informação entre os agentes, funcionalidade adicionais como _logging_ podem ser facilemente implementadas

### Cons
- O mediator pode tornar-se um monolito muito complexo
- Para evitar que a interface do _mediator_ torne-se muito inchada, a comunicação é feita através de mensagens, o que torna as interações menos "estruturadas".

## Discussao:

- Since a Mediator becomes a repository for logic, can the code that implements this logic begin to get overly complex, possible resembling speggheti code? How could this potential problem be solved?